In [1]:
!pip install git+https://github.com/tinygrad/tinygrad.git

  Cloning https://github.com/tinygrad/tinygrad.git to /tmp/pip-req-build-tsrv4387
  Running command git clone --filter=blob:none --quiet https://github.com/tinygrad/tinygrad.git /tmp/pip-req-build-tsrv4387
  Resolved https://github.com/tinygrad/tinygrad.git to commit 745316493ca6a2268c786ecffd12e6df1cbfda0f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for tinygrad: filename=tinygrad-0.9.2-py3-none-any.whl size=1044474 sha256=6f22f1017fa259b70ec1d5dc688a85b90a9014d68a38b0f6664ec7c59c7607d9
  Stored in directory: /tmp/pip-ephem-wheel-cache-b5a4ziek/wheels/8e/aa/2c/100299d3dfaa853eb72242bf26e929f2e3301209c96f9ac9b2
Successfully built tinygrad

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [4]:
from tinygrad import Device
print(Device.DEFAULT)

CLANG


In [13]:
from tinygrad import Tensor, nn
class Model:
  def __init__(self):
    self.l1 = nn.Conv2d(1, 32, kernel_size=(3,3))
    self.l2 = nn.Conv2d(32, 64, kernel_size=(3,3))
    self.l3 = nn.Linear(1600, 10)

  def __call__(self, x:Tensor) -> Tensor:
    x = self.l1(x).relu().max_pool2d((2,2))
    x = self.l2(x).relu().max_pool2d((2,2))
    return self.l3(x.flatten(1).dropout(0.5))

In [4]:
from tinygrad.nn.datasets import mnist
X_train, Y_train, X_test, Y_test = mnist()
print(X_train.shape, X_train.dtype, Y_train.shape, Y_train.dtype)
# (60000, 1, 28, 28) dtypes.uchar (60000,) dtypes.uchar

https://storage.googleapis.com/cvdf-datasets/mnist/train-images-idx3-ubyte.gz: 4
https://storage.googleapis.com/cvdf-datasets/mnist/train-labels-idx1-ubyte.gz: 6
https://storage.googleapis.com/cvdf-datasets/mnist/t10k-images-idx3-ubyte.gz: 7.
https://storage.googleapis.com/cvdf-datasets/mnist/t10k-labels-idx1-ubyte.gz: 10


(60000, 1, 28, 28) dtypes.uchar (60000,) dtypes.uchar


In [28]:
model = Model()
acc = (model(X_test).argmax(axis=1) == Y_test).mean()

In [15]:
print(acc.item())

0.13519999384880066


In [29]:
optim = nn.optim.Adam(nn.state.get_parameters(model))
batch_size = 128
def step():
  Tensor.training = True  # makes dropout work
  samples = Tensor.randint(batch_size, high=X_train.shape[0])
  X, Y = X_train[samples], Y_train[samples]
  optim.zero_grad()
  loss = model(X).sparse_categorical_crossentropy(Y).backward()
  optim.step()
  return loss

# import timeit
# timeit.repeat(step, repeat=5, number=1)

In [31]:
from tinygrad import TinyJit
jit_step = TinyJit(step)

# import timeit
# timeit.repeat(jit_step, repeat=5, number=1)

In [32]:
for step in range(7000):
    loss = jit_step()
    if step%100 == 0:
        Tensor.training = False
        acc = (model(X_test).argmax(axis=1) == Y_test).mean().item()
        print(f"step {step:4d}, loss {loss.item():.2f}, acc {acc*100.:.2f}%")

step    0, loss 38.91, acc 10.58%
step  100, loss 0.27, acc 94.43%
step  200, loss 0.23, acc 96.23%


KeyboardInterrupt: 